# Interview Feedback:

I really liked this speaker. It was interesting to hear fom a Marketing Analyst, as I feel typically I hear from people in hard data science or business, so she was kind of in the middle, which was really interesting to hear and something I am particularly interested in!

In [2]:
import pandas as pd
import duckdb
# establish connection
conn = duckdb.connect('/Users/riadsouza/Downloads/mimic.db', read_only=True)

In [3]:
conn.sql(
    """
    SHOW TABLES;
    """
)

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ ADMISSIONS │
│ D_ICDPROCS │
│ ICUSTAYS   │
│ PATIENTS   │
│ PROCS_ICD  │
└────────────┘

In [4]:
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = 'PROCS_ICD';
    """
)

┌─────────────┬───────────┐
│ column_name │ data_type │
│   varchar   │  varchar  │
├─────────────┼───────────┤
│ row_id      │ INTEGER   │
│ subject_id  │ INTEGER   │
│ hadm_id     │ INTEGER   │
│ seq_num     │ INTEGER   │
│ icd9_code   │ INTEGER   │
└─────────────┴───────────┘

In [11]:
# Problem 1
conn.sql(
    """
    SELECT COUNT(DISTINCT icd9_code) FROM PROCS_ICD;
    """
)

┌───────────────────────────┐
│ count(DISTINCT icd9_code) │
│           int64           │
├───────────────────────────┤
│                       164 │
└───────────────────────────┘

In [6]:
# Problem 2
conn.sql(
    """
    SELECT D_ICDPROCS.short_title, COUNT(PROCS_ICD.icd9_code) FROM D_ICDPROCS
    JOIN PROCS_ICD ON D_ICDPROCS.icd9_code=PROCS_ICD.icd9_code
    GROUP BY short_title
    ORDER BY COUNT(PROCS_ICD.icd9_code) DESC
    LIMIT 5
    """
)

┌──────────────────────────┬────────────────────────────┐
│       short_title        │ count(PROCS_ICD.icd9_code) │
│         varchar          │           int64            │
├──────────────────────────┼────────────────────────────┤
│ Venous cath NEC          │                         56 │
│ Entral infus nutrit sub  │                         32 │
│ Packed cell transfusion  │                         30 │
│ Insert endotracheal tube │                         22 │
│ Cont inv mec ven <96 hrs │                         21 │
└──────────────────────────┴────────────────────────────┘

In [7]:
# Problem 3
conn.sql(
    """
    SELECT ADMISSIONS.subject_id,
    cast(ADMISSIONS.dischtime as date) - cast(ADMISSIONS.admittime as date) as duration
    FROM ADMISSIONS JOIN PROCS_ICD ON ADMISSIONS.subject_id=PROCS_ICD.subject_id
    JOIN D_ICDPROCS ON D_ICDPROCS.icd9_code=PROCS_ICD.icd9_code
    WHERE short_title='Venous cath NEC';
    """
)

┌────────────┬──────────┐
│ subject_id │ duration │
│   int32    │  int64   │
├────────────┼──────────┤
│      10006 │        9 │
│      10011 │       14 │
│      10027 │       12 │
│      10032 │       13 │
│      10036 │        2 │
│      10038 │       12 │
│      10044 │        9 │
│      10059 │        6 │
│      10059 │        7 │
│      10061 │       25 │
│        ·   │        · │
│        ·   │        · │
│        ·   │        · │
│      41976 │        5 │
│      41976 │        7 │
│      41976 │        3 │
│      41976 │        3 │
│      41976 │        3 │
│      41976 │        3 │
│      41976 │        4 │
│      41976 │        3 │
│      41976 │        7 │
│      41976 │        8 │
├────────────┴──────────┤
│  221 rows (20 shown)  │
└───────────────────────┘

In [8]:
conn.sql(
    """
    SELECT MEAN(cast(ADMISSIONS.dischtime as date) - cast(ADMISSIONS.admittime as date)) as avgAge,
    STDDEV(cast(ADMISSIONS.dischtime as date) - cast(ADMISSIONS.admittime as date)) as stdAge
    FROM ADMISSIONS JOIN PROCS_ICD ON ADMISSIONS.subject_id=PROCS_ICD.subject_id
    JOIN D_ICDPROCS ON D_ICDPROCS.icd9_code=PROCS_ICD.icd9_code
    WHERE short_title='Venous cath NEC';
    """
)

┌──────────────────┬───────────────────┐
│      avgAge      │      stdAge       │
│      double      │      double       │
├──────────────────┼───────────────────┤
│ 7.85972850678733 │ 9.860539258651679 │
└──────────────────┴───────────────────┘

In [9]:
# Problem 4
conn.sql(
    """
    SELECT round(mean(cast(PATIENTS.dod as date) - cast(PATIENTS.dob as date))/365, 0) as avgAge,
    round(sum(case when gender = 'M' then 1 else 0 end)/count(*),2) as maleProp,
    round(sum(case when gender = 'F' then 1 else 0 end)/count(*),2) as femProp,
    D_ICDPROCS.short_title
    FROM PATIENTS JOIN ADMISSIONS ON PATIENTS.subject_id=ADMISSIONS.subject_id
    JOIN PROCS_ICD ON ADMISSIONS.subject_id=PROCS_ICD.subject_id
    JOIN D_ICDPROCS ON D_ICDPROCS.icd9_code=PROCS_ICD.icd9_code
    GROUP BY D_ICDPROCS.short_title
    ORDER BY COUNT(PROCS_ICD.icd9_code) DESC
    LIMIT 5;
    """
)

┌────────┬──────────┬─────────┬──────────────────────────┐
│ avgAge │ maleProp │ femProp │       short_title        │
│ double │  double  │ double  │         varchar          │
├────────┼──────────┼─────────┼──────────────────────────┤
│   73.0 │     0.88 │    0.12 │ Venous cath NEC          │
│   66.0 │     0.94 │    0.06 │ Entral infus nutrit sub  │
│   66.0 │     0.88 │    0.12 │ Insert endotracheal tube │
│   72.0 │     0.86 │    0.14 │ Cont inv mec ven <96 hrs │
│   66.0 │      1.0 │     0.0 │ CV cath plcmt w guidance │
└────────┴──────────┴─────────┴──────────────────────────┘